In [21]:
def selectPlayers(pclustered, team, nbats, nbowls, nall):
    pospl = pd.DataFrame()
    
    total_players = nbats + nbowls + nall
    rankbat = list()
    rankbowl = list()
    
    pteam = pclustered[pclustered.cid == team]
    
    lpteam = len(pteam)
    print("lpteam:", lpteam)
    
    if lpteam < total_players:
        print("ERROR: Avaliable players {} is less than required {}".format(lpteam, total_players))
        return None
    
    ptbat = pteam[(pteam.clusterRI == 0) | (pteam.clusterRI == 1)]
    ptbat = ptbat.sort_values(['batRI'], ascending=False).reset_index(drop=True)
    lbat = len(ptbat)
#     print("bats: ", lbat)
    
    ptbowl = pteam[pteam.clusterRI == 4]    
    ptbowl = ptbowl.sort_values(['bowlRI'], ascending=False).reset_index(drop=True)
    lbowl = len(ptbowl)
#     print("lbowl: ", lbowl)
    
    ptall = pteam[(pteam.clusterRI == 2) | (pteam.clusterRI == 3)]
    ptall.insert(2, 'rir', ptall.batRI/ptall.bowlRI)
    lall = len(ptall)
#     print("lall: ", lall)
    
    if lbat < nbats:
        nall = nall + (nbats - lbat)
        nbats = lbat
        
    if lbowl < nbowls:
        nall = nall + (nbowls - lbowl)
        nbowls = lbowl
    
    if lall < nall:
        nbats = nbats + (nall - lall)
        nall = lall
        
    mp = {
        'batsman' : nbats,
        'bowlers': nbowls,
        'allrounders' : nall
    }
    print("Selecting: ", mp)
    
    ptall = ptall.sort_values(['rir', 'batRI','bowlRI'], ascending = False).reset_index(drop=True)
    
    for i in range(0, nbats):
        pospl = pospl.append(ptbat.iloc[i])
    

    for b in range(0, nbowls):
        pospl = pospl.append(ptbowl.iloc[b])
        
    l = len(pospl)
    for b in range(0, nall):
        pospl = pospl.append(ptall.iloc[b])
        
    print("tplayers: ", pospl.shape)
        
#     batting ranking

    batpl = pospl.sort_values(['batRI'], ascending=False).reset_index(drop=True)
    
    
    for b in range(0, total_players):
        rankbat.append((b+1, [batpl.iloc[b]['name'], batpl.iloc[b]['team']]))
        
#     bowling ranking

    bowlpl = pospl.sort_values(['bowlRI'], ascending=False).reset_index(drop=True)
    
    for b in range(0, total_players):
        rankbowl.append((b+1, [bowlpl.iloc[b]['name'], bowlpl.iloc[b]['team']]))
        
    return (rankbat, rankbowl)
        
    

In [2]:
import pandas as pd
import numpy as np

In [26]:
cl = pd.read_csv('clustered055.csv')
players = pd.read_csv('playerList.csv')
ncl = pd.read_csv('npclustered055.csv')
nplayers = pd.read_csv('newPlayerList.csv')

In [27]:
ntplayers = pd.DataFrame.merge(ncl, nplayers, on='pid', how='inner')
ntplayers.insert(7, 'cid', value=5)
ntplayers.head(5)

,pid,batRI,bowlRI,clusterRI,name,yob,team,cid,iscurrent
0,14336.0,-0.260000,1.076885e+00,0,Kuldeep Sen,1998,Madhya Pradesh,5,1
1,10242.0,-0.369895,1.790746e+00,0,Ashish Kumar,1998,Jharkhand,5,1
2,14338.0,-0.130000,-5.054994e-19,0,Madhav Kaushik,1998,Uttar Pradesh,5,1
3,14339.0,-0.130000,-1.092752e-01,0,Moia,1998,MIZORAM,5,1
4,8198.0,0.219787,1.625251e+00,0,Baba Aparajith,1998,Tamil Nadu,5,1


In [28]:
tplayers = pd.DataFrame.merge(cl, players, on='pid', how='inner')
tplayers.insert(6, 'team', value='ODI')
tplayers.head(5)

,pid,batRI,bowlRI,clusterRI,name,yob,team,cid,iscurrent
0,2255.0,-0.000226,6.394588e-01,4,M N Samuels,1981,ODI,10,1
1,2977.0,-0.134686,1.051152e-01,0,H Masakadza,1983,ODI,11,1
2,2995.0,-0.119307,1.741568e+00,2,Mashrafe Mortaza,1983,ODI,3,1
3,3035.0,-0.071118,4.500601e-19,0,P A Patel,1985,ODI,5,1
4,3065.0,-0.293833,3.132003e+00,1,J M Anderson,1982,ODI,4,1


In [32]:
ipl = tplayers[tplayers.cid == 5]
# ipl.sort_values(['batRI'], ascending=False).reset_index(drop=True)

In [15]:
players = tplayers.append(ntplayers, sort=True).reset_index()
players.tail(5)

,index,batRI,bowlRI,cid,clusterRI,iscurrent,name,pid,team,yob
1490,1142,-0.031419,-7.802784e-02,5,0,1,Puneet Yadav,8186.0,Rajasthan,1998
1491,1143,-0.235004,5.073072e-01,5,0,1,Rituraj Singh,8187.0,Goa,1998
1492,1144,-0.151587,-5.054994e-19,5,0,1,Paras Sharma,10236.0,Jammu and Kashmir,1998
1493,1145,-0.236506,0.000000e+00,5,0,1,Neelam Obi,14333.0,ARUNACHALPRADESH,1998
1494,1146,-0.152358,1.968991e+00,5,0,1,Swapnil Singh,10238.0,Baroda,1998


In [16]:
for_country = 5
ipc = players[players['cid']  == for_country]

ipc[ipc.clusterRI == 1]
ipc.shape

(1194, 10)

In [23]:
ranking = selectPlayers(players, team=5, nbats = 5, nbowls = 5, nall = 10)

if ranking is not None:
    print('BATSMAN RANKING')
    print('----------------------------------------------------')
    for pl in ranking[0]:
        print('POSITION: ', "{:>2}: ".format(pl[0]), "{:<20}".format(pl[1][0]), "{:>10}".format("team: "), pl[1][1])

    print()
    print('BOWLER RANKING') 
    print('----------------------------------------------------')
    for pl in ranking[1]:
        print('POSITION: ', "{:>2}: ".format(pl[0]), "{:<20}".format(pl[1][0]), "{:>10}".format("team: "), pl[1][1])


lpteam: 1194
Selecting:  {'batsman': 5, 'bowlers': 5, 'allrounders': 10}
tplayers:  (20, 11)
BATSMAN RANKING
----------------------------------------------------
POSITION:   1:  Prashant Chopra          team:  Himachal Pradesh
POSITION:   2:  Yashpal Singh            team:  Services
POSITION:   3:  Vishnu Vinod             team:  Kerala
POSITION:   4:  Samrath Seth             team:  ARUNACHALPRADESH
POSITION:   5:  Baba Aparajith           team:  Tamil Nadu
POSITION:   6:  Tanmay Agarwal           team:  Hyderabad
POSITION:   7:  Parvez Rasool            team:  ODI
POSITION:   8:  N A Saini                team:  ODI
POSITION:   9:  S N Thakur               team:  ODI
POSITION:  10:  R A Jadeja               team:  ODI
POSITION:  11:  D S Kulkarni             team:  ODI
POSITION:  12:  H H Pandya               team:  ODI
POSITION:  13:  U T Yadav                team:  ODI
POSITION:  14:  K K Ahmed                team:  ODI
POSITION:  15:  A R Patel                team:  ODI
POSITION:  

In [ ]:
### 